# Пример решения некоторых задач курсовой работы с помощью инфраструктуры Python

## Решение задачи линейного программирования

### Условие задачи

Рацион стада крупного рогатого скота включает пищевые продукты A, B, C, D, E. 
В сутки одно животное должно съедать не менее А1 [кг] продукта А, B1 [кг] продукта B, C1 [кг] продукта C, D1 [кг] продукта D, E1 [кг] продукта E. Однако в чистом виде указанные продукты не производятся. 
Они содержатся в кормовых культурах К-1, K-2, K-3, K-4. Известно долевое содержание каждого продукта в каждой культуре. 
Известны суточные нормы A1-E1 (в кг). 
Определить максимальное поголовье скота, которое можно содержать в течение года, используя имеющийся запас культур.

### Решение

#### Параметры
Будем использовать $i$ для индексирования продуктов, $j$ - для индексирования культур.

Содержание продукта $i$ в культуре $j$ - $r_{ij}$ ($i \in {1..5}, j \in {1..4}, r_{ij} \in [0..1]$).

Суточная норма продукта $i$ (на одно животное) - $d_i, i \in {1..5}$ \[кг\].

Запас культуры $j$ - $s_j$ \[т\].

#### Переменные

$n$ – количество животных.

$x_j \in {1..4}$ – количество культуры $j$-того типа, которую мы используем в кормовом плане \[т\].

#### Формальная постановка

$$
n \rightarrow max \\
x_1 r_{11} + x_2 r_{12} + x_3 r_{13} + x_4 r_{14} \geq 0.365d_1n \\
\dots \\
x_1 \leq s_1 \\
x_i, n \geq 0
$$

#### Решение в Python

https://cvxopt.org/userguide/coneprog.html#cvxopt.solvers.lp


In [1]:
from cvxopt import matrix, solvers

c = matrix([0, 0, 0, 0, -1], tc='d')                       # -1, потому что по умолчанию решается задача минимизации
G = matrix([[-0.05,-0.13,    0,    0,  0.365 * 1.5],       # Матрица с коэффициентами ограничений-неравенств
            [-0.10,-0.20,-0.10,    0,  0.365 * 1.4],       # Нам пришлось домножить строки на -1, чтобы представить
            [-0.04,-0.06,-0.15,    0,  0.365 * 1.0],       # ограничения вида >=
            [   0, -0.10,-0.08,-0.15,  0.365 * 0.8],
            [-0.15,    0,-0.14,-0.10,  0.365 * 0.6],
            [1, 0, 0, 0, 0],                               # Непревышение запасов
            [0, 1, 0, 0, 0],                               # x_i <= s_i
            [0, 0, 1, 0, 0],
            [0, 0, 0, 1, 0],
            [-1, 0, 0, 0, 0],                              # Неотрицательность переменных
            [ 0,-1, 0, 0, 0],                              # x_i >= 0
            [ 0, 0,-1, 0, 0],                              # следовательно,
            [ 0, 0, 0,-1, 0],                              # -x_i <= 0    (поэтому коэффициенты равны -1)
            [ 0, 0, 0, 0,-1],
           ], tc='d')
h = matrix([0, 0, 0, 0, 0, 5, 6, 7, 8, 0, 0, 0, 0, 0], tc='d')

solution = solvers.lp(c, G.T, h, solver='glpk')
print('Status: ', solution['status'])
print('x = \n', solution['x']) # Значения переменных
print('Objective: ', solution['primal objective'])

Status:  optimal
x = 
 [ 5.00e+00]
[ 6.00e+00]
[ 8.44e-01]
[ 0.00e+00]
[ 1.88e+00]

Objective:  -1.881278538812786


Курсовая работа

Условия задачи:

x <= 179 ( x = x11 + x12 + x13 + x21 + x22 + x23 )

x12 = 0.8 * x11
x13 = 0.8 * x12
x22 = 0.62 * x21
x23 = 0.62 * x22
x11 + x12 + x13 + x21 + x22 + x23 <= 179
xij >= 0


f = 5 + (x11 + 40) ^ (1/3) + ... + 7 + (x21 + 30)^1/3 + ... -> max